In [1]:
import pandas as pd
import numpy as np
import json
import pickle
import random
from tqdm import tqdm

In [2]:
speakers_floor_speeches_bids = open('../floor_speeches_congs_115_116/clean/author_map.txt').readlines()

In [3]:
speakers_floor_speeches_bids = list(map(lambda x:x.rstrip(), speakers_floor_speeches_bids))
print(len(speakers_floor_speeches_bids))

513


In [4]:
raw_data = pd.read_csv('all_tweets_df.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3445918 entries, 0 to 3445917
Data columns (total 4 columns):
Tweet ID     int64
Author ID    int64
Text         object
Timestamp    object
dtypes: int64(2), object(2)
memory usage: 105.2+ MB
None


In [5]:
bid_to_twitter_uid = pickle.load(open('social_media_data_bioguide_to_twitter.pkl', 'rb'))
print(len(bid_to_twitter_uid))

530


In [6]:
## first we remove all tweets from twitter authors for whom we are not calculating floor speech TBIPs
relev_twitter_uids = []
for bid in speakers_floor_speeches_bids:
    if bid in bid_to_twitter_uid:
        relev_twitter_uids.append(bid_to_twitter_uid[bid])
relev_twitter_uids = sorted(set(relev_twitter_uids))
print(len(relev_twitter_uids))

502


In [7]:
raw_data = raw_data[raw_data['Author ID'].isin(relev_twitter_uids)]
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2667283 entries, 4 to 3445678
Data columns (total 4 columns):
Tweet ID     int64
Author ID    int64
Text         object
Timestamp    object
dtypes: int64(2), object(2)
memory usage: 101.7+ MB
None


In [8]:
#remove speakers if they gave tweeted less than 100 times
speakers_to_remove_based_on_num_tweets = set()
speakers = set(raw_data['Author ID'])
thresh = 100
for s in speakers:
    n_s = len(raw_data[raw_data['Author ID']==s])
    if n_s < thresh:
        speakers_to_remove_based_on_num_tweets.add(s)
print(len(speakers_to_remove_based_on_num_tweets))

0


In [14]:
# use stopwords used for speeches and also combine stopwords provided by TBIP authors that they used for senate tweets
stopwords = open('../floor_speeches_congs_115_116/stopwords.txt').readlines()
stopwords = list(map(lambda x:x.rstrip(), stopwords))
print(len(stopwords))

1456


In [15]:
stopwords_sen_tweets = open('../../setup/stopwords/senate_tweets.txt').readlines()
stopwords_sen_tweets = list(map(lambda x:x.rstrip(), stopwords_sen_tweets))
print(len(stopwords_sen_tweets))

505


In [16]:
stopwords = stopwords + stopwords_sen_tweets
print(len(stopwords))

1961


In [13]:
rep_accounts_115 = pd.read_csv('representatives-accounts-1.csv')
print(rep_accounts_115.info())
rep_accounts_115.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 3 columns):
Token    435 non-null object
Uid      435 non-null int64
Link     435 non-null object
dtypes: int64(1), object(2)
memory usage: 10.3+ KB
None


,Token,Uid,Link
0,KYComer,838462994,https://twitter.com/KYComer
1,RepJackyRosen,818554054309715969,https://twitter.com/RepJackyRosen
2,RepEspaillat,817076257770835968,https://twitter.com/RepEspaillat
3,RepTrey,811986281177772032,https://twitter.com/RepTrey
4,RepDwightEvans,90639372,https://twitter.com/RepDwightEvans


In [17]:
stopwords = stopwords + list(map(lambda x:x.lower(), list(rep_accounts_115['Token'])))
print(len(stopwords))

2396


In [18]:
rep_accounts_116 = pd.read_csv('congress116-house-accounts.csv')
print(rep_accounts_116.info())
rep_accounts_116.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 3 columns):
Token    760 non-null object
Uid      760 non-null int64
Link     760 non-null object
dtypes: int64(1), object(2)
memory usage: 17.9+ KB
None


,Token,Uid,Link
0,RepDevinNunes,344972339,https://twitter.com/RepDevinNunes
1,AngieCraigMN,411861905,https://twitter.com/AngieCraigMN
2,benraylujan,91125308,https://twitter.com/benraylujan
3,bradyfortexas,570005456,https://twitter.com/bradyfortexas
4,chiproytx,1257667158,https://twitter.com/chiproytx


In [19]:
stopwords = stopwords + list(map(lambda x:x.lower(), list(rep_accounts_116['Token'])))
print(len(stopwords))

3156


In [20]:
stopwords = set(stopwords)
print(len(stopwords))

2460


In [21]:
f = open('stopwords.txt', 'w')
for i, x in enumerate(list(stopwords)):
    f.write(x)
    if i < len(stopwords) - 1:
        f.write('\n')
f.close()

In [22]:
#rest of the preprocessing is roughly follows the script provided in the TBIP repo - setup/senate_tweets_to_bag_of_words.py 
import os

import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
#below, speeches mean tweets
speakers = list(raw_data['Author ID'])
print(len(speakers))
speeches = list(raw_data['Text'])
print(len(speeches))

2667283
2667283


In [24]:
speaker_to_speaker_id = dict(
    [(y, x) for x, y in enumerate(sorted(set(speakers)))])
author_indices = np.array(
    [speaker_to_speaker_id[s] for s in speakers])
print(len(author_indices))
author_map = np.array(list(speaker_to_speaker_id.keys()))
print(len(author_map))

2667283
471


In [25]:
#data is too big as it stands, randomly sample tweets so as to preserve original distribution of tweets-per-author
author_to_inds = {}

sampled_N = 300000

N = len(speeches)
for i, a in tqdm(enumerate(speakers)):
    if a in author_to_inds:
        author_to_inds[a].append(i)
    else:
        author_to_inds[a] = [i]
        
selected_inds = []
for a in author_to_inds:
    l = author_to_inds[a]
    random.Random(1).shuffle(l)
    perc = round(sampled_N*(len(l)/N))
    selected_inds = selected_inds + l[:perc]
    
print(len(selected_inds))

2667283it [00:01, 1495474.80it/s]


300006


In [26]:
#selected_inds = sorted(selected_inds)
raw_data = raw_data.iloc[selected_inds]
print(raw_data.info())
raw_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300006 entries, 349114 to 3191035
Data columns (total 4 columns):
Tweet ID     300006 non-null int64
Author ID    300006 non-null int64
Text         300006 non-null object
Timestamp    300006 non-null object
dtypes: int64(2), object(2)
memory usage: 11.4+ MB
None


,Tweet ID,Author ID,Text,Timestamp
349114,996837967976194048,818554054309715969,The skyrocketing cost of life-saving medicatio...,2018-05-16 19:41:06
55260,878358077145325569,818554054309715969,To all of the brave women who have had to say ...,2017-06-23 21:04:19
152423,979935734701412353,818554054309715969,#Passover is a beautiful time to remember our ...,2018-03-31 04:17:40
109698,1040330596545904643,818554054309715969,I was glad to help pass bipartisan legislation...,2018-09-13 20:05:17
162220,923648345394040833,818554054309715969,GOP budget does nothing to help Nevada's middl...,2017-10-26 20:31:40


In [27]:
raw_data.to_csv('sampled_twitter_data_raw.csv', index=False)

In [28]:
#below, speeches mean tweets
speakers = list(raw_data['Author ID'])
print(len(speakers))
speeches = list(raw_data['Text'])
print(len(speeches))

300006
300006


In [29]:
speaker_to_speaker_id = dict(
    [(y, x) for x, y in enumerate(sorted(set(speakers)))])
author_indices = np.array(
    [speaker_to_speaker_id[s] for s in speakers])
print(len(author_indices))
author_map = np.array(list(speaker_to_speaker_id.keys()))
print(len(author_map))

300006
471


In [30]:
count_vectorizer = CountVectorizer(min_df=0.0005,
                                   max_df=0.75, 
                                   stop_words=stopwords, 
                                   ngram_range=(1, 3),
                                   token_pattern="[a-zA-Z#]+")
# Learn initial document term matrix. This is only initial because we use it to
# identify words to exclude based on author counts.
counts = count_vectorizer.fit_transform(speeches)
vocabulary = np.array(
    [k for (k, v) in sorted(count_vectorizer.vocabulary_.items(), 
                            key=lambda kv: kv[1])])


/workspace/.conda/envs/tbip/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abby', 'allard', 'andr', 'andybiggs', 'anthonybrownmd', 'auctnr', 'austinscottga', 'balart', 'balderson', 'barrag', 'bettymccollum', 'clyburnsc', 'col', 'colon', 'conawaytx', 'congressnm', 'cortez', 'davejoyceoh', 'deb', 'delgadoforny', 'desjarlaistn', 'dr', 'drmarkgreen', 'drnealdunnfl', 'dutch', 'garc', 'gonz', 'guti', 'halleran', 'jackbergman', 'jacksonleetx', 'jasoncrowco', 'jes', 'jos', 'jr', 'katiehill', 'kellyforms', 'labrador', 'lacyclaymo', 'lamarsmithtx', 'lehtinen', 'lez', 'louiegohmerttx', 'luj', 'markamodeinv', 'maxrose', 'mikesimpson', 'mucarsel', 'nchez', 'ocasio', 'powell', 'ra', 'ratcliffetx', 'raulruiz', 'rdenas', 'rep', 'replancenj', 'ros', 'rourke', 'roybal', 'rrez', 'sfv', 'shea', 'spanbergerva', 'sr', 'steveknight', 'stevewomack', 'teamgt', 'tomcoleok',

In [31]:
print(counts.shape)
print(len(vocabulary))

(300006, 3512)
3512


In [32]:
author_to_inds = {}
for a in tqdm(list(author_map)):
    inds = []
    author_ind = speaker_to_speaker_id[a]
    for i, ind in enumerate(list(author_indices)):
        if ind==author_ind:
            inds.append(i)
    author_to_inds[a] = inds

100%|██████████| 471/471 [00:55<00:00,  8.56it/s]


In [33]:
def get_per_author_counts(counts, author_to_inds):
    list_of_arrays = []
    for a in author_to_inds:
        inds = author_to_inds[a]
        list_of_arrays.append(np.array(np.sum(counts[inds], 0)))#.reshape((1, counts.shape[1])))
    return np.concatenate(list_of_arrays, 0)

In [34]:
counts_per_author = get_per_author_counts(counts, author_to_inds)
print(counts_per_author.shape)

(471, 3512)


In [35]:
# Remove phrases spoken by less than 5 representatives
min_authors_per_word = 5

In [36]:
acceptable_words = []
for i in range(len(vocabulary)):
    if np.count_nonzero(counts_per_author[:, i]) >= min_authors_per_word:
        acceptable_words.append(i)
print(len(acceptable_words))

3509


In [37]:
count_vectorizer = CountVectorizer(ngram_range=(1, 3),
                                   vocabulary=vocabulary[acceptable_words])
counts = count_vectorizer.fit_transform(speeches)
vocabulary = np.array(
    [k for (k, v) in sorted(count_vectorizer.vocabulary_.items(), 
                            key=lambda kv: kv[1])])

In [38]:
print(counts.shape)
print(len(vocabulary))

(300006, 3509)
3509


In [39]:
# `n_gram_to_unigram` takes as key an index to an n-gram in the vocabulary
# and its value is a list of the vocabulary indices of the corresponding 
# unigrams.
n_gram_indices = np.where(
  np.array([len(word.split(' ')) for word in vocabulary]) > 1)[0]
n_gram_to_unigrams = {}
for n_gram_index in n_gram_indices:
    matching_unigrams = []
    for unigram in vocabulary[n_gram_index].split(' '):
        if unigram in vocabulary:
            matching_unigrams.append(np.where(vocabulary == unigram)[0][0])
    n_gram_to_unigrams[n_gram_index] = matching_unigrams

# `n_grams_to_bigrams` now breaks apart trigrams and higher to find bigrams 
# as subsets of these words.
n_grams_to_bigrams = {}
for n_gram_index in n_gram_indices:
    split_n_gram = vocabulary[n_gram_index].split(' ')
    n_gram_length = len(split_n_gram) 
    if n_gram_length > 2:
        bigram_matches = []
        for i in range(0, n_gram_length - 1):
            bigram = " ".join(split_n_gram[i:(i + 2)])
            if bigram in vocabulary:
                bigram_matches.append(np.where(vocabulary == bigram)[0][0])
        n_grams_to_bigrams[n_gram_index] = bigram_matches

In [40]:
# Go through counts, and remove a unigram each time a bigram superset 
# appears. Also remove a bigram each time a trigram superset appears.
# Note this isn't perfect: if bigrams overlap (e.g. "global health care" 
# contains "global health" and "health care"), we count them both. This
# may introduce a problem where we subract a unigram count twice, so we also
# ensure non-negativity.
#counts_dense = counts.toarray()
for i in tqdm(range(counts.shape[0])):
    n_grams_in_doc = np.where(counts[i, n_gram_indices].toarray() > 0)[0]
    sub_n_grams = n_gram_indices[n_grams_in_doc]
    for n_gram in sub_n_grams:
        counts[i, n_gram_to_unigrams[n_gram]] = sparse.csr_matrix(counts[i, n_gram_to_unigrams[n_gram]].toarray() - counts[i, n_gram])
        if n_gram in n_grams_to_bigrams:
            counts[i, n_grams_to_bigrams[n_gram]] = sparse.csr_matrix(counts[i, n_grams_to_bigrams[n_gram]].toarray() - counts[i, n_gram])
counts[counts < 0] = 0

100%|██████████| 300006/300006 [02:09<00:00, 2315.36it/s]


In [41]:
print(counts.shape)

(300006, 3509)


In [42]:
# Remove speeches with no words.
existing_speeches = []#np.where(np.sum(counts_dense, axis=1) > 0)[0]
for i in tqdm(range(counts.shape[0])):
    if counts[i].sum() > 0:
        existing_speeches.append(i)
counts = counts[existing_speeches]
print(counts.shape)
author_indices = author_indices[existing_speeches]
print(author_indices.shape)

100%|██████████| 300006/300006 [01:23<00:00, 3601.91it/s]


(294078, 3509)
(294078,)


In [43]:
# Save data.

# `counts.npz` is a [num_documents, num_words] sparse matrix containing the
# word counts for each document.
sparse.save_npz("clean/counts.npz",
                counts.astype(np.float32))

# `author_indices.npy` is a [num_documents] vector where each entry is an
# integer indicating the author of the corresponding document.
np.save("clean/author_indices.npy", author_indices)

# `vocabulary.txt` is a [num_words] vector where each entry is a string
# denoting the corresponding word in the vocabulary.
np.savetxt("clean/vocabulary.txt", vocabulary, fmt="%s")

# `author_map.txt` is a [num_authors] vector of strings providing the bioguide ID of
# each author in the corpus.
np.savetxt("clean/author_map.txt", author_map, fmt="%s")

In [44]:
# `raw_documents.txt` contains all the documents we ended up using.
raw_documents = [document.replace("\n", ' ').replace("\r", ' ') 
                 for i, document in tqdm(enumerate(speeches)) if i in existing_speeches]
print(len(raw_documents))

300006it [10:48, 462.64it/s]

294078


In [45]:
f = open('clean/raw_documents.txt', 'w')
for i, doc in enumerate(raw_documents):
    f.write(doc)
    if i < len(raw_documents) - 1:
        f.write('\n')

In [46]:
f.close()

In [47]:
raw_data = raw_data.iloc[existing_speeches]
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294078 entries, 349114 to 3191035
Data columns (total 4 columns):
Tweet ID     294078 non-null int64
Author ID    294078 non-null int64
Text         294078 non-null object
Timestamp    294078 non-null object
dtypes: int64(2), object(2)
memory usage: 11.2+ MB
None


In [48]:
raw_data.to_csv('finalized_tbip_tweets_sampled.csv', index=False)

In [49]:
print('Done')

Done
